In [1]:
import numpy as np
import pandas as pd
import pickle
import os
import pdb
from tqdm import tqdm

import scipy.stats as stats
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
import seaborn as sns

import joblib
from joblib import Parallel, delayed

from gimmemotifs.motif import read_motifs
from gimmemotifs.scanner import Scanner
from gimmemotifs.fasta import Fasta
from gimmemotifs.motif.denovo import gimme_motifs

os.environ["XDG_CACHE_HOME"] = "/clusterfs/nilah/aniketh/gimmemotifs"

np.random.seed(97)

/global/scratch/users/aniketh/mamba_envs/gimme/lib/python3.10/site-packages/tqdm/autonotebook.py:19: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  warn(WARN_NOIPYW, TqdmWarning, stacklevel=2)


In [2]:
final_design_seqs_dir = "/global/scratch/users/aniketh/promoter_modelling/jax_data/final_design/"
ensemble_predictions_dir = "/global/scratch/users/aniketh/promoter_modelling/jax_data/ensemble_predictions/"

In [3]:
methods = ["coms", "dens"]
num_seqs = [None, None]
all_seqs = []
all_sources = []
for i, method in enumerate(methods):
    print(method, num_seqs[i])
    seqs = pickle.load(open(os.path.join(final_design_seqs_dir, f"{method}_final_design.pkl"), "rb"))
    seqs_df = []
    kmer_features = {}
    for cell in ["jurkat", "k562", "thp1"]:
        kmer_features[cell] = seqs[cell]["kmer_features"]
        seqs[cell].pop("kmer_features")
        df = pd.DataFrame(seqs[cell])
        df["designed_for"] = cell
        
        if num_seqs[i] is not None:
            df = df.tail(num_seqs[i])
        
        seqs_df.append(df)
    
    seqs_df = pd.concat(seqs_df).reset_index(drop=True)
    all_seqs.extend(seqs_df["sequences"])
    all_sources.extend([f"{method}" for s in seqs_df["sequences"]])
    
    break
    
all_sources = np.array(all_sources)
print(len(all_seqs))
print(len(all_sources))

coms None
15000
15000


In [4]:
def create_temp_fasta(seqs, file_path):
    with open(file_path, "w+") as f:
        for i, seq in enumerate(seqs):
            f.write(f">{i}\n")
            f.write(f"{seq}\n")

In [5]:
fasta_path = "temp.fa"
create_temp_fasta(all_seqs, fasta_path)
outdir = "temp_gimme"
params = {
    "tools": "Homer,BioProspector",
    "genome": "hg38"
    }

motifs = gimme_motifs(fasta_path, outdir, params=params)

2023-07-24 19:22:04,808 - INFO - starting full motif analysis
2023-07-24 19:22:04,812 - INFO - using size of 200, set size to 0 to use original region size
2023-07-24 19:22:04,813 - INFO - preparing input from FASTA
2023-07-24 19:22:05,607 - INFO - Creating index for genomic GC frequencies.


NotImplementedError: "windowMaker" does not appear to be installed or on the path, so this method is disabled.  Please install a more recent version of BEDTools and re-import to use this method.

In [ ]:
from gimmemotifs.scanner import Scanner

In [ ]:
s = Scanner()

In [ ]:
s.set_genome("hg38")
s.set_threshold(fpr=0.01)